In [3]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains

m = wt.watertap_setup(dynamic = False)


wt.case_study_trains.train = {"case_study": "tampa_bay",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "tampa_bay", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "seawater"}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.


tampa_bay
------- Adding Unit Processes -------
sw_onshore_intake
sulfuric_acid_addition
ferric_chloride_addition
chlorination
static_mixer
tri_media_filtration
cartridge_filtration
ro_first_pass
ro_second_pass
lime_softening
chlorination_b
caustic_soda_addition
ammonia_addition
treated_storage
21600.0
backwash_solids_handling
municipal_drinking
surface_discharge
-------------------------------------
adding source
adding splitter
params into splitter --> [0.67, 0.33]
----- Connecting Unit Processes -----
seawater ToUnitName --> sw_onshore_intake
sw_onshore_intake ToUnitName --> sulfuric_acid_addition
sulfuric_acid_addition ToUnitName --> ferric_chloride_addition
ferric_chloride_addition ToUnitName --> chlorination
chlorination ToUnitName --> static_mixer
tri_media_filtration ToUnitName --> cartridge_filtration
tri_media_filtration ToUnitName --> backwash_solids_handling
lime_softening ToUnitName --> chlorination_b
chlorination_b ToUnitName --> caustic_soda_addition
caustic_soda_additio

In [4]:
wt.display.show_train2(model_name=m)

In [5]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [6]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("unfixing feed presure and area for", key)
        
wt.run_water_tap(m = m, objective=True)

# prints RO results
print("optimal ro area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print("setting feed presure for", key, "to -->", m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])

wt.run_water_tap(m = m, objective=True, print_model_results="summary")

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

unfixing feed presure and area for ro_first_pass
unfixing feed presure and area for ro_second_pass
degrees_of_freedom: 4
WaterTAP3 solution optimal
optimal ro area and pressures:
ro_first_pass feed pressure 58.34633458170394
ro_first_pass membrane area 149292.29208671325
ro_second_pass feed pressure 89.9999984749248
ro_second_pass membrane area 21315.732816039417
setting feed presure for ro_first_pass to --> 85
setting feed presure for ro_second_pass to --> 35
degrees_of_freedom: 2
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 10.216867778290633
----------------------------------------------------------------------
fs.sulfuric_acid_addition
total_cap_investment: 0.23072715211599915
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 2.9536394454284336
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 9.265881440407075
------------------

In [7]:
# If you need the system recovery to match better.... set a maximum recovery rate.

In [8]:
from pyomo.environ import Constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.55) # THIS IS FOR TAMPA BAY
#m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.50) # THIS IS FOR SANTA BARBARA

In [9]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 0.75

In [10]:
wt.run_water_tap(m = m, objective=True, print_model_results="summary")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

degrees_of_freedom: 2
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 10.216867778290633
----------------------------------------------------------------------
fs.sulfuric_acid_addition
total_cap_investment: 0.23072715211599915
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 2.9536394454284336
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 9.265881440407075
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0.1162155841035918
----------------------------------------------------------------------
fs.tri_media_filtration
total_cap_investment: 8.067722349162832
----------------------------------------------------------------------
fs.cartridge_filtration
total_cap_investment: 7.584516536765473
----------------------------------------------------------------------
fs.ro_first_pa

In [11]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [12]:
# Readjust recovery constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)

In [13]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=False, print_model_results="summary")

degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 10.216867778290633
----------------------------------------------------------------------
fs.sulfuric_acid_addition
total_cap_investment: 0.23072715211599915
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 2.9536394454284336
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 9.265881440407075
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0.1162155841035918
----------------------------------------------------------------------
fs.tri_media_filtration
total_cap_investment: 8.067722349162832
----------------------------------------------------------------------
fs.cartridge_filtration
total_cap_investment: 7.584516536765473
----------------------------------------------------------------------
fs.ro_first_pa

In [14]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])

In [15]:
# sensitivity analyses
sens_df = pd.DataFrame()

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []

lcow_list.append(value(m.fs.costing.LCOW))
water_recovery_list.append(value(m.fs.costing.system_recovery))
scenario_value.append("n/a")
scenario_name.append("baseline")
elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))

runs_per_scenario = 50

############ onstream_factor 70-100% ############
stash_value = m.fs.costing_param.plant_cap_utilization
scenario = "plant_cap_utilization"
print(scenario)
ub = 1
lb = 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.costing_param.plant_cap_utilization = i
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_cap_utilization = stash_value    
############################################################    

############ salinity  30-45 ############
stash_value = m.fs.seawater.conc_mass_in[0, "tds"]()
scenario = "inlet salinty 25k-45k mg/L"
print(scenario)
ub = 25
lb = 45
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.seawater.conc_mass_in[0, "tds"].fix(i) 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.conc_mass_in[0, "tds"].fix(stash_value) 
############################################################

############ inlet flow ############
stash_value = m.fs.seawater.flow_vol_in[0]()
scenario = "inlet flow"
print(scenario)
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.seawater.flow_vol_in.fix(i) 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.flow_vol_in.fix(stash_value) 
############################################################

############ lifetime years ############

stash_value = value(m.fs.costing_param.plant_lifetime_yrs)
scenario = "lifetime (yrs)"
print(scenario)
ub = 50
lb = 15
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.costing_param.plant_lifetime_yrs = i 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_lifetime_yrs = stash_value
############################################################

############ elec cost +-20% ############

stash_value = value(m.fs.costing_param.electricity_price)
scenario = "electricity price +- 20%"
print(scenario)
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.costing_param.electricity_price = i 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.electricity_price = stash_value
############################################################

############ RO scenarios --> pressure, membrane area, replacement rate% ############

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        area = value(getattr(m.fs, key).membrane_area[0])

        scenario_dict = {"membrane_area" : [-area*0.2, area*0.2], 
                         "pressure": [-10, 10], 
                         "factor_membrane_replacement": [-0.1, 0.1]}
        
        for scenario in scenario_dict.keys():
            if scenario == "pressure":
                stash_value = value(getattr(getattr(getattr(m.fs, key), "feed"), scenario)[0])
            else:
                stash_value = value(getattr(getattr(m.fs, key), scenario)[0])

            print(scenario)
            ub = stash_value + scenario_dict[scenario][1]
            lb = stash_value + scenario_dict[scenario][0]
            step = (ub - lb) / runs_per_scenario
            
            for i in np.arange(lb, ub, step):
                if scenario == "pressure":
                    getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(i)
                else:
                    getattr(getattr(m.fs, key), scenario).fix(i)
                
                wt.run_water_tap(m = m, objective=False)
                print("LCOW -->", m.fs.costing.LCOW())
    
                lcow_list.append(value(m.fs.costing.LCOW))
                water_recovery_list.append(value(m.fs.costing.system_recovery))
                scenario_value.append(i)
                scenario_name.append(key + "_" + scenario)
                elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
            if scenario == "pressure":
                getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(stash_value)
            else:
                getattr(getattr(m.fs, key), scenario).fix(stash_value)
                            
############################################################

# final run to get baseline numbers again
wt.run_water_tap(m = m, objective=False)

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["elec_lcow_difference"] = (sens_df.elec_lcow - value(m.fs.costing.elec_frac_LCOW))

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100


plant_cap_utilization
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.3055822583116472
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2944866583826526
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.283578062946844
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2728517838692934
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2623032884228627
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.251928192901579
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.241722256546403
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2316813757657044
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.221801578633894
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2120790196527227
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2025099747607275
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1930908365772233
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 

WaterTAP3 solution optimal
LCOW --> 1.180219847052989
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1814394861603084
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1827607997588152
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.184180982708117
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.185697360483698
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1873073851254905
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.189008632034289
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1907987977158356
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1926756983667781
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1946372692783818
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.196681565032388
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.198806760470999
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2010111524289317
degrees_of_freedom: 0
Water

degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.161589322048526
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1663354986353878
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1710816752222502
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.175827851809112
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1805740283959743
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1853202049828362
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1900663815696986
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1948125581565603
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1995587347434225
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2043049113302844
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2090510879171465
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2137972645040085
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2185434410908706
d

LCOW --> 1.2242023949906808
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2231144765047341
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2220842676418213
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2211118075702476
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2201972712835893
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2193409845291658
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.218543441094571
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2178053227420325
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2171275220778408
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2165111686147847
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2159576582132108
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2154686859471329
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2150462822026673
degrees_of_freedom: 0
WaterTAP3 solution optimal


degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2838280497602714
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2773949402051183
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2711197440276416
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2650266761482523
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2591478154648714
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2535250964685216
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2482118545133534
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2432725517883327
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2387783940861383
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2347965518395054
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2313735925866285
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2285196665886762
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.226203314010328


WaterTAP3 solution optimal
LCOW --> 1.2144089497600046
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.214684582515396
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.214960215270787
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.215235848026178
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2155114807815688
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2157871135369598
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2160627462923512
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2163383790477418
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.216614011803133
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.216889644558524
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2171652773139152
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2174409100693062
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2177165428246974
degrees_of_freedom: 0
Wate

In [17]:
sens_df.to_csv("results/case_studies/tampa_bay_baseline_sensitivity.csv")

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)